In [1]:
import os

import itertools
import pickle
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import math 

from torchtext import data, datasets
from pathlib import Path

import sys
sys.path.append('../')
#import utils
#import wiki_utils
%matplotlib inline

In [2]:
# Init cuda
device = "cuda:6" if torch.cuda.is_available() else "cpu"
idevice = 6 if torch.cuda.is_available() else -1
torch.cuda.set_device(idevice)

In [3]:
#corpus = wiki_utils.Texts('./wikitext/')
TEXT = data.Field(sequential=True, tokenize=lambda x : list(x))
PATH = Path('./wikitext/')

In [4]:
batch_size = 128
sequence_length = 30
grad_clip = 0.1
lr = 4.
best_val_loss = None
log_interval = 1000

In [5]:
train_ds, valid_ds, test_ds = datasets.WikiText2.splits(TEXT, root=PATH)
#train_loader = wiki_utils.TextLoader(corpus.train, batch_size=batch_size)
#val_loader = wiki_utils.TextLoader(corpus.valid, batch_size=eval_batch_size)
#test_loader = wiki_utils.TextLoader(corpus.test, batch_size=eval_batch_size)

In [6]:
TEXT.build_vocab(train_ds)

In [7]:
eval_batch_size = 128
train_loader, val_loader, test_loader = data.BPTTIterator.splits((train_ds, valid_ds, test_ds), 
                                        batch_sizes=(batch_size, eval_batch_size, eval_batch_size), 
                                        bptt_len=sequence_length, repeat=False, device=idevice) 
#wiki_utils.TextLoader(corpus.train, batch_size=batch_size)
#wiki_utils.TextLoader(corpus.valid, batch_size=eval_batch_size) 
#wiki_utils.TextLoader(corpus.test, batch_size=eval_batch_size)

In [8]:
class RNNModel(nn.Module):

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, hidden=None):
        emb = self.drop(self.encoder(x))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (weight.new(self.nlayers, bsz, self.nhid).zero_(),
                    weight.new(self.nlayers, bsz, self.nhid).zero_())
        else:
            return weight.new(self.nlayers, bsz, self.nhid).zero_()

In [9]:
def evaluate(data_loader):
    model.eval()
    total_loss = 0
    ntokens = len(TEXT.vocab) #len(corpus.dictionary)
    hidden = model.init_hidden(eval_batch_size)
    for i, b in enumerate(data_loader):
        data, targets = b.text, b.target
        output, hidden = model(data)
        output_flat = output.view(-1, ntokens)
        total_loss += len(data) * criterion(output_flat, targets.view(-1)).item()
    return total_loss / len(data_loader)

In [10]:
def train():
    model.train()
    total_loss = 0
    ntokens = len(TEXT.vocab) #len(corpus.dictionary)
    for batch, b in enumerate(train_loader):
        data, targets = b.text, b.target
    #for batch, i in enumerate(range(0, train_data.size(0) - 1, sequence_length)):
        #data, targets = get_batch(train_data, i)
        model.zero_grad()
        output, hidden = model(data)
        loss = criterion(output.view(-1, ntokens), targets.view(-1))
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_loader), lr, cur_loss, math.exp(cur_loss)))
            total_loss = 0

In [11]:
ntokens = len(TEXT.vocab) #len(corpus.dictionary)
model = RNNModel('LSTM', ntokens, 128, 128, 2, 0.3).to(device)
criterion = nn.CrossEntropyLoss()

In [12]:
def generate(n=50, temp=1.):
    model.eval()
    x = torch.rand(1, 1).mul(ntokens).long().to(device)
    hidden = None
    out = []
    for i in range(n):
        output, hidden = model(x, hidden)
        s_weights = output.squeeze().data.div(temp).exp()
        s_idx = torch.multinomial(s_weights, 1)[0]
        x.data.fill_(s_idx)
        s = TEXT.vocab.itos[s_idx] #corpus.dictionary.idx2symbol[s_idx]
        out.append(s)
    return ''.join(out)

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
%%time
with torch.no_grad():
    print('sample:\n', generate(50), '\n')

for epoch in range(1, 51):
    train()
    val_loss = evaluate(val_loader)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')


sample:
 LTń♭ვჯრÅწāGoن=.Ü>ë7صリリL,*ო3đ﻿–1áდò²!†戦iv>áīXyøAプ჻— 

| epoch   1 |  1000/ 2808 batches | lr 4.00 | loss  3.12 | ppl    22.62
| epoch   1 |  2000/ 2808 batches | lr 4.00 | loss  2.44 | ppl    11.45
-----------------------------------------------------------------------------------------
| end of epoch   1 | valid loss 58.52 | valid ppl 26109660363823811915677696.00
-----------------------------------------------------------------------------------------
sample:
  the Sphlartaar of on the Boflent andore bysurat a 

| epoch   2 |  1000/ 2808 batches | lr 4.00 | loss  2.06 | ppl     7.85
| epoch   2 |  2000/ 2808 batches | lr 4.00 | loss  1.96 | ppl     7.10
-----------------------------------------------------------------------------------------
| end of epoch   2 | valid loss 50.20 | valid ppl 6332017721268421787648.00
-----------------------------------------------------------------------------------------
sample:
  becine yeat find to the roo approse Grees . The < 

| epoch   

| epoch  19 |  1000/ 2808 batches | lr 4.00 | loss  1.59 | ppl     4.93
| epoch  19 |  2000/ 2808 batches | lr 4.00 | loss  1.59 | ppl     4.91
-----------------------------------------------------------------------------------------
| end of epoch  19 | valid loss 41.54 | valid ppl 1101952905907838464.00
-----------------------------------------------------------------------------------------
sample:
  . <eos> The regular stable months was conception exam 

| epoch  20 |  1000/ 2808 batches | lr 4.00 | loss  1.59 | ppl     4.91
| epoch  20 |  2000/ 2808 batches | lr 4.00 | loss  1.59 | ppl     4.90
-----------------------------------------------------------------------------------------
| end of epoch  20 | valid loss 41.47 | valid ppl 1023033142110551424.00
-----------------------------------------------------------------------------------------
sample:
 an indostression and top . The south , comparation 

| epoch  21 |  1000/ 2808 batches | lr 4.00 | loss  1.59 | ppl     4.90
| epoc

| epoch  37 |  1000/ 2808 batches | lr 1.00 | loss  1.56 | ppl     4.75
| epoch  37 |  2000/ 2808 batches | lr 1.00 | loss  1.56 | ppl     4.74
-----------------------------------------------------------------------------------------
| end of epoch  37 | valid loss 40.65 | valid ppl 452277387784495296.00
-----------------------------------------------------------------------------------------
sample:
 labames , when by the genus last of the Torru , Go 

| epoch  38 |  1000/ 2808 batches | lr 1.00 | loss  1.56 | ppl     4.75
| epoch  38 |  2000/ 2808 batches | lr 1.00 | loss  1.56 | ppl     4.74
-----------------------------------------------------------------------------------------
| end of epoch  38 | valid loss 40.64 | valid ppl 447746471814796736.00
-----------------------------------------------------------------------------------------
sample:
 orating them would installed on the zayvers , is a 

| epoch  39 |  1000/ 2808 batches | lr 1.00 | loss  1.56 | ppl     4.75
| epoch  39 

In [15]:
t1 = generate(10000, 1.)
t15 = generate(10000, 1.5)
t075 = generate(10000, 0.75)
with open('./generated075.txt', 'w') as outf:
    outf.write(t075)
with open('./generated1.txt', 'w') as outf:
    outf.write(t1)
with open('./generated15.txt', 'w') as outf:
    outf.write(t15)